In [1]:
# Set the layers, variables and parameters

import os
import math
import arcpy
import numpy as np
import concurrent.futures
from arcpy import env
env.parallelProcessingFactor = "100%"  # use all available cores
env.overwriteOutput = True

In [2]:
# Setting Layers

dir = r"C:\Capstone\ConnectivityAnalyses"
study_area = os.path.join(dir,"ShelbyROI.shp")
hexagons = os.path.join(dir,"hexagons.shp")
patches_layer = os.path.join(dir,"patchROI.shp")


In [ ]:
# Setting coordinate system

# Define target projected coordinate system
projected_crs = arcpy.SpatialReference(26916)  # NAD 1983 UTM Zone 16N (meters)

# Create a folder for projected data
projected_dir = os.path.join(dir, "Projected")
os.makedirs(projected_dir, exist_ok=True)

# Define projected file paths
study_area_proj = os.path.join(projected_dir, "ShelbyROI_UTM16N.shp")
patches_layer_proj = os.path.join(projected_dir, "patchROI_UTM16N.shp")
hexagons_proj = os.path.join(projected_dir, "hexagons_UTM16N.shp")

# Project original layers into UTM Zone 16N (meters)
arcpy.management.Project(study_area, study_area_proj, projected_crs)
arcpy.management.Project(patches_layer, patches_layer_proj, projected_crs)
arcpy.management.Project(hexagons, hexagons_proj, projected_crs)

study_area = study_area_proj
patches_layer = patches_layer_proj
hexagons = hexagons_proj


In [4]:
# Setting variables

# Set the variable for the hexagons area field
area_hexagons = "Area_m2"
area_patches = "Area_m2"

# Set the variable for the hexagons ID field
ID_field = "GRID_ID"

# Set the variable for the name of the pc index shapefile
hexagons_pc = f"{hexagons[:-4]}_pc.shp"

In [5]:
# Setting parameters

distance = 500 # Distance threshold
pij = 0.5 # Probability of dispersal
radius = distance*4
area_hex = 3*math.sqrt(3)/2*radius**2 # Area calculated using radius 3 times the mean dispersal distance

In [6]:
# Generate the hexagons grid shapefile

hexagons_temp = os.path.join(dir,"hexagons_temp.shp") 

arcpy.ba.GenerateGridsAndHexagons(
    area_of_interest=study_area,
    out_feature_class=hexagons_temp, # Uses the full path defined above
    cell_type="HEXAGON",
    enrich_type="",
    cell_size=f"{area_hex} SquareMeters",
    h3_resolution=7,
    variables=None,
    distance_type="STRAIGHT_LINE_DISTANCE",
    distance=1,
    units="MILES",
    out_enriched_buffers=None,
    travel_direction="TOWARD_STORES",
    time_of_day=None,
    time_zone="TIME_ZONE_AT_LOCATION",
    search_tolerance=None,
    polygon_detail="STANDARD",
    out_centroids=None
)

# Select hexagons that intersect (touch or fall within) the study area
arcpy.management.MakeFeatureLayer(hexagons_temp, "hex_layer")
arcpy.management.SelectLayerByLocation(
    in_layer="hex_layer",
    overlap_type="INTERSECT",  # keeps any hexagon that touches or overlaps
    select_features=study_area,
    selection_type="NEW_SELECTION"
)

# Export the selected hexagons (unclipped) to 'hexagons'
arcpy.management.CopyFeatures("hex_layer", hexagons)

# Define spatial reference
desc_study_area = arcpy.Describe(study_area)
spatial_reference = desc_study_area.spatialReference

# Define type for area_hexagons attribute field
arcpy.AddField_management(hexagons, area_hexagons, "DOUBLE")

# Calculate the area of the hexagons
arcpy.management.CalculateGeometryAttributes(
    in_features=hexagons,
    geometry_property="Area_m2 AREA",
    length_unit="",
    area_unit="SQUARE_METERS",
    coordinate_system=spatial_reference,
    coordinate_format="SAME_AS_INPUT"
    )


<Result 'C:\\Capstone\\ConnectivityAnalyses\\Projected\\hexagons_UTM16N.shp'>

In [7]:
# Define the function to calculate the probability of connectivity index (PC index) for a given set of patches
# and a given distance threshold, following Saura & Pascual-Hortal (2007).

def PC_index(patches, area_field, total_area, distance, pij):
    """
    Function to calculate the probability of connectivity index (PC index) for a given set of patches,
    following Saura & Pascual-Hortal (2007).
    Parameters:
    - patches(str):
        Path to the patches shapefile
    - area_field(str):
        Name of the field containing the area of the patches
    - total_area(float):
        Area of the study area
    - distance(float):
        Distance threshold (in meters)
    - pij(float):
        Probability of dispersal at the median distance
    Returns:
    - pc_index_global(float):
        Probability of connectivity index (PC index) for the given set of patches
    """
    import arcpy
    import math

    # decay constant k
    k = -math.log(pij) / distance

    # calculate areas for patches (if not already present)
    arcpy.management.CalculateGeometryAttributes(
        in_features=patches,
        geometry_property=f"{area_field} AREA",
        area_unit="SQUARE_METERS"
    )

    # collect patch areas
    patch_areas = {}
    with arcpy.da.SearchCursor(patches, ["OID@", area_field]) as cur:
        for oid, area in cur:
            patch_areas[oid] = area

    # generate near table (pairwise distances)
    near_table = f"{patches[:-4]}_near.dbf"
    arcpy.analysis.GenerateNearTable(
        in_features=patches,
        near_features=patches,
        out_table=near_table,
        location="NO_LOCATION",
        angle="NO_ANGLE",
        closest="ALL",
        method="PLANAR",
        distance_unit="Meters"
    )

    # add self-links (distance = 0)
    with arcpy.da.SearchCursor(patches, ["OID@"]) as s:
        with arcpy.da.InsertCursor(near_table, ["IN_FID", "NEAR_FID", "NEAR_DIST"]) as ins:
            for (oid,) in s:
                ins.insertRow((oid, oid, 0.0))

    # compute numerator: ΣΣ a_i * a_j * exp(-k * d_ij)
    numerator = 0.0
    with arcpy.da.SearchCursor(near_table, ["IN_FID", "NEAR_FID", "NEAR_DIST"]) as cur:
        for i, j, dist in cur:
            numerator += patch_areas[i] * patch_areas[j] * math.exp(-k * dist)

    # final PC
    pc_global = numerator / (total_area ** 2)
    return pc_global

In [9]:
#Step 1. Calculate the pc index for each hexagon using the patches shapefile

# Copy the hexagons shapefile to a new shapefile
arcpy.management.CopyFeatures(hexagons, hexagons_pc)

# Add the fields to the new shapefile
arcpy.AddField_management(hexagons_pc, "PC", "DOUBLE")
arcpy.AddField_management(hexagons_pc, "ratio", "DOUBLE")
arcpy.AddField_management(hexagons_pc, "Threshold", "TEXT")

# Create a new folder to store the patches for each hexagon
dir_patches = os.path.join(dir, "PatchesHex")
os.makedirs(dir_patches, exist_ok=True)  # Create the folder if it doesn't exist

with arcpy.da.UpdateCursor(hexagons_pc, [ID_field, area_hexagons, "PC", "ratio","Threshold"]) as cursor:
    for row in cursor:
        grid_id, area, pc_value, ratio_area, threshold = row
        where_clause = f"{ID_field} = '{grid_id}'"
        target_hex = arcpy.SelectLayerByAttribute_management(hexagons, "NEW_SELECTION", where_clause)
        patches_hex = os.path.join(dir_patches, f"{grid_id}.shp")

        # Clip patches to the current hexagon
        arcpy.analysis.PairwiseClip(patches_layer, target_hex, patches_hex)

        # Check if there are patches in this hexagon
        count_result = arcpy.management.GetCount(patches_hex)
        count = int(count_result.getOutput(0))

        if count == 0:
            pc_value = 0
            ratio_area = 0
            threshold = "Low"
            print(f"Hexagon {grid_id} has no patches. Setting PC to 0.")
        else:
            # Ensure Area_m2 field exists
            fields = [f.name for f in arcpy.ListFields(patches_hex)]
            if area_patches not in fields:
                arcpy.AddField_management(patches_hex, area_patches, "DOUBLE")

            # Calculate the area for the patches in this hexagon
            desc_patches = arcpy.Describe(patches_hex)
            sr_patches = desc_patches.spatialReference

            arcpy.management.CalculateGeometryAttributes(
                in_features=patches_hex,
                geometry_property=[[area_patches, "AREA"]],
                area_unit="SQUARE_METERS",
                coordinate_system=sr_patches
            )

            # Compute PC for the hexagon
            pc_value = PC_index(
                patches=patches_hex,
                area_field=area_patches,
                total_area=area,
                distance=distance,
                pij=pij
            )

            # Calculate the patches_area/hexagon_area ratio
            with arcpy.da.SearchCursor(patches_hex, [area_patches]) as patches_cursor:
                patches_area = sum(row[0] for row in patches_cursor)
            ratio_area = patches_area / area

            # Classify based on ratio
            if ratio_area > 0.6:
                threshold = "Biodiversity source"
            elif ratio_area < 0.2:
                threshold = "Low"
            else:
                threshold = "Intermediate"

        # Update fields
        row[2] = pc_value
        row[3] = ratio_area
        row[4] = threshold
        cursor.updateRow(row)



Hexagon C-15 has no patches. Setting PC to 0.
Hexagon E-15 has no patches. Setting PC to 0.
Hexagon G-15 has no patches. Setting PC to 0.
Hexagon I-15 has no patches. Setting PC to 0.
Hexagon K-15 has no patches. Setting PC to 0.
Hexagon A-13 has no patches. Setting PC to 0.
Hexagon D-13 has no patches. Setting PC to 0.
Hexagon C-12 has no patches. Setting PC to 0.
Hexagon L-11 has no patches. Setting PC to 0.
Hexagon E-10 has no patches. Setting PC to 0.
Hexagon F-10 has no patches. Setting PC to 0.
Hexagon J-10 has no patches. Setting PC to 0.
Hexagon M-10 has no patches. Setting PC to 0.
Hexagon G-9 has no patches. Setting PC to 0.
Hexagon W-8 has no patches. Setting PC to 0.
Hexagon F-7 has no patches. Setting PC to 0.
Hexagon W-7 has no patches. Setting PC to 0.
Hexagon W-6 has no patches. Setting PC to 0.
Hexagon E-5 has no patches. Setting PC to 0.
Hexagon W-5 has no patches. Setting PC to 0.
Hexagon G-4 has no patches. Setting PC to 0.
Hexagon W-4 has no patches. Setting PC to 

In [10]:
print(arcpy.Describe(patches_hex).spatialReference.name)

NAD_1983_UTM_Zone_16N


In [11]:
# This script merges hexagons classified as "Biodiversity source" and recalculates the area of the merged hexagons.

# Set the name for the temporary shapefile to store the dissolved hexagons based on the classification

hexagons_dissolved = os.path.join(dir,"hexagons_dissolved.shp") #temp

# Dissolve by classification
hexagons_dissolved = os.path.join(dir, "hexagons_dissolved.shp")
arcpy.analysis.PairwiseDissolve(
    in_features=hexagons_pc, 
    out_feature_class=hexagons_dissolved, 
    dissolve_field="Threshold", 
    multi_part="SINGLE_PART"
)

# Export only the Biodiversity source hexagons
biodiversity_source = os.path.join(dir, "biodiversity_source.shp")
arcpy.analysis.Select(
    in_features=hexagons_dissolved,
    out_feature_class=biodiversity_source,
    where_clause="Threshold = 'Biodiversity source'"
)

# Erase them from the full set
hexagons_erased = os.path.join(dir, "hexagons_erased.shp")
arcpy.analysis.Erase(
    in_features=hexagons_pc,
    erase_features=biodiversity_source,
    out_feature_class=hexagons_erased
)

# Merge erased + biodiversity source back together
hexagons_merged = os.path.join(dir, "hexagons_merged.shp")
arcpy.management.Merge(
    inputs=[hexagons_erased, biodiversity_source],
    output=hexagons_merged,
    field_mappings=None,
    add_source="NO_SOURCE_INFO"
)

# Select the hexagons that has no GRID_ID. This is the case of the hexagons dissolved that are classified as "Biodiversity source"
hexagons_nulls = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=hexagons_merged,
    selection_type="NEW_SELECTION",
    where_clause="GRID_ID = ' '",
    invert_where_clause=None
)

# Set a new GRID_ID for the hexagons that has no GRID_ID
arcpy.management.CalculateField(
    in_table=hexagons_nulls,
    field="GRID_ID",
    expression="'Z-' + str(!FID!)",
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

# Clear the selection
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=hexagons_nulls,
    selection_type="CLEAR_SELECTION"
)

# Get the spatial reference of the hexagons shapefile

desc = arcpy.Describe(hexagons_nulls)
SR = desc.spatialReference

# Calculate the new area for final hexagons

arcpy.management.CalculateGeometryAttributes(
    in_features=hexagons_nulls,
    geometry_property="Area_m2 AREA",
    length_unit="",
    area_unit="SQUARE_METERS",
    coordinate_system=SR,
    coordinate_format="SAME_AS_INPUT"
)

# Change the variable for the next steps
hexagons_pc = os.path.join(dir,"hexagons_pc_classified.shp")

# Create the final output shapefile
# Use hexagons_merged as the input
arcpy.management.CopyFeatures(hexagons_merged, hexagons_pc)

#Erase temporal layers
arcpy.management.Delete(hexagons_dissolved)
arcpy.management.Delete(hexagons_erased)
arcpy.management.Delete(hexagons_merged) # delete this temp file


<Result 'true'>

In [13]:
# Re-calculate the PC index for each FL (hexagon) using the patches shapefile and the area field

import time

dir_patches2 = os.path.join(dir, "PatchesHex2")
os.makedirs(dir_patches2, exist_ok=True)

total_start = time.time()
counter = 0

with arcpy.da.UpdateCursor(hexagons_pc, [ID_field, area_hexagons, "PC", "ratio", "Threshold"]) as cursor:
    for row in cursor:
        grid_id, area, pc_value, ratio_area, threshold = row
        if threshold != "Biodiversity source":
            continue  # only process relevant hexagons

        counter += 1
        print(f"\n=== Processing hexagon {grid_id} (#{counter}) ===")
        start_time = time.time()

        where_clause = f"{ID_field} = '{grid_id}'"
        target_hex = arcpy.SelectLayerByAttribute_management(hexagons_pc, "NEW_SELECTION", where_clause)
        patches_hex = os.path.join(dir_patches2, f"{grid_id}.shp")

        # Clip patches to the current hexagon
        arcpy.analysis.PairwiseClip(patches_layer, target_hex, patches_hex)

        # If there are no patches in the hexagon, skip
        count_result = arcpy.management.GetCount(patches_hex)
        count = int(count_result.getOutput(0))
        if count == 0:
            pc_value = 0
            ratio_area = 0
            print(f"    No patches found → PC = 0")
        else:
            # Compute PC for the hexagon (fast version)
            pc_value = PC_index(
                patches=patches_hex,
                area_field=area_patches,
                total_area=area,
                distance=distance,
                pij=pij
            )

            # Compute area ratio
            patches_area = 0
            with arcpy.da.SearchCursor(patches_hex, ["Area_m2"]) as patches_cursor:
                for patches_row in patches_cursor:
                    patches_area += patches_row[0]
            ratio_area = patches_area / area

        # Update fields
        row[2] = pc_value
        row[3] = ratio_area
        cursor.updateRow(row)

        elapsed = time.time() - start_time
        print(f"    Done in {elapsed:.2f} seconds — PC={pc_value:.6f}, ratio={ratio_area:.3f}")

total_elapsed = time.time() - total_start
print(f"\n✅ Finished all hexagons in {total_elapsed/60:.2f} minutes.")


=== Processing hexagon Z-252 (#1) ===
    Done in 268.73 seconds — PC=0.280669, ratio=0.864

=== Processing hexagon Z-253 (#2) ===
    Done in 13.11 seconds — PC=0.211757, ratio=0.613

✅ Finished all hexagons in 4.70 minutes.


In [14]:
# Priorization of restoration

import networkx as nx
from itertools import combinations

# ─── PARAMETERS ───────────────────────────────────────────────────────────────

pc_field    = "PC"
class_field = "Threshold"

# Define which class-values you want for connector analysis:
INTERMEDIATE_CLASSES = ["Intermediate"]  # e.g. ["MedRes", "IntRes"]
SOURCE_CLASSES       = ["Biodiversity source"]      # e.g. ["Source"]

connector_classes = set(INTERMEDIATE_CLASSES + SOURCE_CLASSES)

out_fields = ["varPCflux", "varPCconn", "Priority"]

In [15]:
# 1. Make layer for spatial queries
lyr = "FL_lyr"
arcpy.MakeFeatureLayer_management(hexagons_pc, lyr)

# 2. Build graph G of ALL FLs (nodes carry PC & Classification)
G = nx.Graph()
with arcpy.da.SearchCursor(lyr, [ID_field, pc_field, class_field]) as cur:
    for oid, pc, cl in cur:
        G.add_node(oid, PC=float(pc), Classification=cl)

# 3. Add edges for immediate neighbors
arcpy.MakeFeatureLayer_management(hexagons_pc, "FL_main")
arcpy.MakeFeatureLayer_management(hexagons_pc, "FL_compare")


for oid, in arcpy.da.SearchCursor("FL_main", [ID_field]):
    # select just that hexagon in the main layer
    arcpy.SelectLayerByAttribute_management(
        "FL_main", "NEW_SELECTION", f"{ID_field} = '{oid}'"
    )
    # find all hexagons in FL_compare that touch it
    arcpy.SelectLayerByLocation_management(
        "FL_compare", "BOUNDARY_TOUCHES", "FL_main",
        "", "NEW_SELECTION"
    )
    # add edges for each neighbor found
    with arcpy.da.SearchCursor("FL_compare", [ID_field]) as nbr_cur:
        for nbr_oid, in nbr_cur:
            if nbr_oid != oid:
                G.add_edge(oid, nbr_oid)
    # clear selections before next iteration
    arcpy.SelectLayerByAttribute_management("FL_main", "CLEAR_SELECTION")
    arcpy.SelectLayerByAttribute_management("FL_compare", "CLEAR_SELECTION")


In [16]:
# 4. Calculate varPCflux
# 4.1 Create a dictionary with the centroids of the hexagons and their PC values
ids, pc_vals = [], []
centroids = {}
with arcpy.da.SearchCursor(hexagons_pc, [ID_field, "PC", "SHAPE@XY"]) as cur:
    for oid, pc, xy in cur:
        ids.append(oid)
        pc_vals.append(pc)
        centroids[oid] = xy

n = len(ids)
pc_array = np.array(pc_vals, dtype=float)
# 4.2 Create a distance matrix
dist_matrix = np.zeros((n, n), dtype=float)
for i in range(n):
    x1, y1 = centroids[ids[i]]
    for j in range(n):
        x2, y2 = centroids[ids[j]]
        dist_matrix[i, j] = math.hypot(x1 - x2, y1 - y2)

# 4.3 Decay constant k and p*ij matrix
k = - math.log(pij) / distance # decay constant
p_matrix = np.exp(-k * dist_matrix) # p*ij matrix

# 4.4 Total flux (i≠j)
flux_total = 0.0
for i in range(n):
    for j in range(n):
        if i == j: 
            continue
        flux_total += pc_array[i] * pc_array[j] * p_matrix[i, j]

# 4.5 varPCflux for each hexagon
var_flux = {}
for idx, oid in enumerate(ids):
    # máscara para excluir idx
    mask = np.ones(n, dtype=bool)
    mask[idx] = False
    pc_mask = pc_array[mask]
    p_mask = p_matrix[np.ix_(mask, mask)]

    # flux without idx
    flux_mask = 0.0
    nm = n - 1
    for i in range(nm):
        for j in range(nm):
            if i == j: 
                continue
            flux_mask += pc_mask[i] * pc_mask[j] * p_mask[i, j]

    var_flux[oid] = (flux_total - flux_mask) / flux_total if flux_total > 0 else 0.0

In [17]:
# 5 Normalize both to [0,1] and sum → Priority
def normalize(d):
    vals = list(d.values())
    lo, hi = min(vals), max(vals)
    if hi == lo:
        return {k: 0.0 for k in d}
    return {k: (v - lo) / (hi - lo) for k, v in d.items()}


# 5.1 Create a dictionary with the PC values of all nodes
# (this is needed for the varPCconnector calculation)
all_pc = {n: G.nodes[n]['PC'] for n in G.nodes()}

# --- 1) Define nodes “conector” ---
connector_nodes = [
    n for n,d in G.nodes(data=True)
    if d["Classification"] in connector_classes
]

# --- 2) Calculate PC_connector global ---
# all_pc ya es {nodo: PC_nodo}
# components of connector subgraph
S = G.subgraph(connector_nodes)
comps = list(nx.connected_components(S))

PC_conn_total = 0.0
for c1, c2 in combinations(comps, 2):
    s1 = sum(all_pc[n] for n in c1)
    s2 = sum(all_pc[n] for n in c2)
    PC_conn_total += 2 * s1 * s2

# --- 3) For each node, experiment with removing ---
var_conn = {}
for node in connector_nodes:
    # Nodes without own node
    nodes_minus = [n for n in connector_nodes if n != node]
    S_minus = G.subgraph(nodes_minus)
    comps_minus = list(nx.connected_components(S_minus))
    
    PC_conn_minus = 0.0
    for c1, c2 in combinations(comps_minus, 2):
        s1 = sum(all_pc[n] for n in c1)
        s2 = sum(all_pc[n] for n in c2)
        PC_conn_minus += 2 * s1 * s2

    # Relative variation
    var_conn[node] = (
        (PC_conn_total - PC_conn_minus) / PC_conn_total
        if PC_conn_total > 0 else 0.0
    )


# Normalize flux and connector values to [0,1]
norm_flux = normalize(var_flux)
norm_conn = normalize(var_conn)

In [18]:
# 6. Function to calculate priority according to Tambosi 2014
def calculate_priority_tambosi(n,norm_flux, norm_conn, classification):
    """Calcula prioridad según Tambosi 2014"""
    if classification == "Intermediate":
        return norm_flux[n] + norm_conn[n]
    elif classification == "Low":
        return 0.0
    else:  # Biodiversity source
        return 0.0

# 6.1 Calculate priority for each node
priority_tambosi = {
    n: calculate_priority_tambosi(n, norm_flux, norm_conn, G.nodes[n]['Classification'])
    for n in G.nodes
}

# 6.2 Normalize priority to [0,1]
norm_priority = normalize(priority_tambosi)

In [19]:
# 7. Write results to hexagons shapefile
existing = [f.name for f in arcpy.ListFields(hexagons_pc)]
for fld in out_fields:
    if fld not in existing:
        arcpy.AddField_management(hexagons_pc, fld, "DOUBLE")

with arcpy.da.UpdateCursor(hexagons_pc, [ID_field] + out_fields) as ucur:
    for row in ucur:
        oid = row[0]
        row[1] = norm_flux.get(oid, 0.0)
        row[2] = norm_conn.get(oid, 0.0)
        row[3] = norm_priority.get(oid, 0.0)
        ucur.updateRow(row)

print("Finished writing results to hexagons shapefile.")

Finished writing results to hexagons shapefile.
